## B3情報科学演習：時系列データ分析（後編）

ここでは, data.ipynbで処理を施したデータ(processed_data.csv)を用いて, 学習モデルを作成していく  
今回は, RandomForestとLSTMという2つの学習モデルを実装・評価を行う  
一言で, 学習モデルと言っても, 学習モデルは多数存在しており, その裏で動くアルゴリズムも異なる  
更に言えば, 同じモデル名でもレイヤーの構成・パラメータ等でも性能に違いが出てくる  

分からない単語やモデル名, コード等は逐一調べて, ふんわりでも良いので理解していくことが重要  
また, あくまでこれは参考用に記述してモノであり, 厳密に言うと不適切だったり, 工夫の余地がまだまだ存在していたりするので, こういうやり方なんだなぁ程度で読んでいくのがおすすめ  

---

### ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
import math

# Random Forest用
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

# LSTM用
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import scipy.stats

# 警告フィルター
import warnings
warnings.filterwarnings('ignore')

---

### 〇 Random Forest

Random Forestは決定木ベースのアンサンブル（バギング）モデル  
決定木ベースなので完全な連続値の表現はできないが，複数の決定木により可能になる

#### データ読み込み

In [ ]:
trainval_data = pd.read_csv('./data/processed_data.csv', index_col=['UNIX_Time'])
trainval_data.head()

#### 関数定義

In [ ]:
#評価指標算出用
result_score=['R^2', 'RMSE']
def get_score(Y_true, Y_pred):
    r2 = r2_score(Y_true, Y_pred)
    rmse = mean_squared_error(Y_true, Y_pred, squared=False)
    return round(r2, 3), round(rmse, 2)

#### 学習準備

In [ ]:
#目的変数を指定
target_name = 'Radiation'

#説明変数と目的変数に分割
X_trainval_data, Y_trainval_data = trainval_data.drop([target_name], axis=1), trainval_data.loc[:,target_name]

#学習データと検証データに分割
X_train, X_validation, Y_train, Y_validation = train_test_split(X_trainval_data, Y_trainval_data, train_size=0.8, shuffle=False)

#### パラメータ探索

基本的にどんな学習器でも手動で決定するパラメータ（ハイパーパラメータ）が存在する  
訓練・検証データを使って交差検証の結果から適したハイパーパラメータを見つける  
チューニングもやり方が色々あるが，一番単純なグリッドサーチを使う  

In [ ]:
#パラメータ候補
params_list = {
    'max_depth' : [5, 6, 7],
    'max_features': ['auto', None],
    'n_estimators': [110, 120],
    'random_state' : [0] #乱数固定用(探索不要)
}

In [ ]:
# GridSearchのクラス定義
grid_search = GridSearchCV(RandomForestRegressor(), params_list, cv=3, verbose=1)

# 探索開始
grid_search.fit(X_trainval_data, Y_trainval_data)

# ベストパラメータ出力
print("最良パラメータ: {}".format(grid_search.best_params_))
print("最良交差検証スコア: {:.2f}".format(grid_search.best_score_))

# ベストパラメータを取得
best_max_depth = grid_search.best_params_.get('max_depth')
best_max_features = grid_search.best_params_.get('max_features')
best_n_estimators = grid_search.best_params_.get('n_estimators')

#### 学習・検証

In [ ]:
# シード値の固定用
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(11)
random.seed(101)
tf.random.set_seed(1001)

# モデル構築
model = RandomForestRegressor(
    max_depth=best_max_depth, 
    max_features=best_max_features, 
    n_estimators=best_n_estimators,
    )

#学習実行
model.fit(X_train, Y_train)
pred_train = model.predict(X_train)

#検証実行
pred_validation = model.predict(X_validation)

#### 評価

In [ ]:
train_score = pd.DataFrame([get_score(Y_train, pred_train)], columns=result_score, index=['Train'])
validation_score = pd.DataFrame([get_score(Y_validation, pred_validation)], columns=result_score, index=['Val'])

df_result = pd.concat([train_score, validation_score], axis=0)
df_result

#### 学習・検証予測線の描画

In [ ]:
# 訓練データの予測線
cmap = plt.get_cmap('tab10')
plt.figure(figsize=[16.5,3.5])
plt.plot(Y_train.index, Y_train, c=cmap(0), linewidth=1.5, label='True')
plt.plot(Y_train.index, pred_train, c=cmap(1), linewidth=1.5, label='Predict')
plt.xticks(['2016-09-01 09:35:11-10:00', '2016-09-22 12:05:21-10:00', '2016-10-11 23:45:18-10:00', '2016-10-30 05:30:18-10:00', '2016-11-17 08:10:04-10:00'])
plt.title('Train Predict')
plt.xlabel('UNIX_Time')
plt.ylabel('Radiation')
plt.legend()
plt.tight_layout()

In [ ]:
# 検証データの予測線
plt.figure(figsize=[16.5,3.5])
plt.plot(Y_validation.index, Y_validation, c=cmap(0), linewidth=1.5, label='True')
plt.plot(Y_validation.index, pred_validation, c=cmap(2), linewidth=1.5, label='Predict')
plt.xticks(['2016-11-17 08:15:02-10:00', '2016-11-21 20:50:02-10:00', '2016-11-26 09:40:20-10:00', '2016-12-02 02:55:04-10:00', '2016-12-09 05:55:52-10:00'])
plt.title('Validation Predict')
plt.xlabel('UNIX_Time')
plt.ylabel('Radiation')
plt.legend()
plt.tight_layout()

#### 特徴量重要度(おまけ)

Random Forestを使用した特徴量重要度を見てみる  
決定木ベースのモデル（scikit-learn）は基本的に重要度が見れるようになっている  
藤浪「どうやって重要度を求めてるのかは知らないので割愛」  
B4原田「調べておきました」  

<img src='./image/image2.jpg' width=600>

実際には検証データに対して重要度を見て，不要な特徴量を除去するなど対処を行う  
テストデータで見るのは自由だが，その結果から特徴量エンジニアリング等をするのは不正

In [ ]:
feature_importance = pd.Series(index=X_train.columns, data=model.feature_importances_)
feature_importance.sort_values(inplace=True)

plt.figure(figsize=[13,5])
plt.xlabel('Feature')
plt.ylabel('Importance Score')
plt.title('Feature Importance')
plt.bar(feature_importance.index, feature_importance)

---

### 〇 LSTM：Long Term Short Memory

LSTM（Long Short Term Memory）はニューラルネットワークの一種で時系列データの学習を得意とするモデル  
RNNと呼ばれるニューラルネットワークの一種でもあるが，RNNよりも長い時系列に対応するように設計されている  

#### データ読み込み

In [ ]:
data = pd.read_csv('./data/processed_data.csv').values
data = data[:,1:]
data = np.asarray(data).astype(np.float64)

#### 関数定義

LSTMや1D-CNNなどの時系列モデルではサブシーケンス(ウィンドウ)形式でデータを入力する必要がある  
サブシーケンスは部分系列を意味しており，一定期間のデータを1つのデータとしたもの  

何時点前までのデータを見るかを決め，スライディングウィンドウ方式でデータを作成していく  
ちなみに見る時点の範囲をルックバック数といい，この数はデータのドメイン知識から決めることがベターだと思う

In [ ]:
#LSTM用のデータ整形
def generate_data(data, input_data, output_data, sequence_size):
    X_data=[]
    Y_data=[]

    #入力シーケンスサイズ
    look_back = sequence_size

    #入力シーケンス分だけ繰り返す
    for i in range(data.shape[0]-look_back):
        X_timedata=[]
        Y_timedata=[]

        if i+look_back+1 < data.shape[0]:
            #サブキャリア分だけ繰り返す
            for j in range(input_data.shape[1]):
            #i~i+lookbackはシーケンス範囲, jは特徴量インデックス
                X_timedata.append(input_data[i:i+look_back, j])
            X_timedata = np.array(X_timedata)
            X_timedata = X_timedata.transpose()
            X_data.append(X_timedata)

            #目的変数は1つのみなのでシーケンス分だけ繰り返す
            Y_timedata.append(output_data[i+look_back-1])
            Y_timedata = np.array(Y_timedata)
            Y_timedata = Y_timedata.transpose()
            Y_data.append(Y_timedata)

    X_data = np.array(X_data)
    Y_data = np.array(Y_data)

    return X_data, Y_data

In [ ]:
#学習・検証曲線の描画
def make_graph(i, batch_size_list, history):
    plt.figure(figsize=[6,3])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Batch_Size : ' + str(batch_size_list[i]))
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.tight_layout()

In [ ]:
#テストデータ予測用(今は実行できない)
def LSTM_test_predict(best_model, sequence_size):
    test_data = pd.read_csv('./test/test_data.csv').values
    test_data = test_data[:,1:]
    test_data = np.asarray(test_data).astype(np.float64)

    input_data = test_data[:,1:]
    input_data = scipy.stats.zscore(input_data)
    output_data = test_data[:,0]

    X_test, Y_test = generate_data(test_data, input_data, output_data, sequence_size)
    
    pred_test = best_model.predict(X_test)
    test_score = pd.DataFrame([get_score(Y_test, pred_test)], columns=result_score, index=['Test'])

    cmap = plt.get_cmap('tab10')
    plt.figure(figsize=[16.5,3.5])
    plt.plot(range(0, len(Y_test)), Y_test, c=cmap(0), linewidth=1.5, label='True')
    plt.plot(range(0, len(pred_test)), pred_test, c=cmap(3), linewidth=1.5, label='Predict')
    plt.title('Test Predict')
    plt.xlabel('UNIX_Time Index')
    plt.ylabel('Radiation')
    plt.legend()
    plt.tight_layout()

    return pred_test, test_score

#### 学習準備

ニューラルネットワークを使用する際はデータを正規化，標準化することとが定石になっている  
正規化することで学習速度を向上させることができる  
理由としては，各データはスケールが異なる（単位が異なる）ことから元々の状態で予測値に対する影響度が違うため  
値が大きいと誤差逆伝播により重み更新をする際に移動量が大きくなり，収束に時間がかかる

In [ ]:
#目的変数と説明変数に分割
input_data = data[:,1:]
input_data = scipy.stats.zscore(input_data) #説明変数のみを標準化
output_data = data[:,0]

#LSTM用にデータ加工
sequence_size = 10
X_trainval_data, Y_trainval_data = generate_data(data, input_data, output_data, sequence_size)

#モデル構築用_変数設定
X_dim = X_trainval_data.shape[2]
Y_dim = Y_trainval_data.shape[1]

#学習用_変数設定
epochs_max = 100
validation_split_rate = 0.2
verbose_level = 0
batch_size_min = 16
batch_size_max = 64
batch_size_list = []
for a in range(int(math.log2(batch_size_min))-1, int(math.log2(batch_size_max))):
    batch_size_list.append(2**(a+1))

#データ分割(学習用と検証用)
X_train = X_trainval_data[:-int(len(X_trainval_data)*(validation_split_rate))]
Y_train = Y_trainval_data[:-int(len(Y_trainval_data)*(validation_split_rate))]
X_validation = X_trainval_data[-int(len(X_trainval_data)*(validation_split_rate)):]
Y_validation = Y_trainval_data[-int(len(Y_trainval_data)*(validation_split_rate)):]

#変数の初期化
best_r2 = -float('inf')
best_rmse = float('inf')

#### 学習・検証

深層学習のフレームワークにはtensorflowとPytorchが有名  
ライブラリで多少学習結果が変わるらしいがネットワーク構成は似たように書けるはずなので好きな方を使ってください  
※今回は, tensorflowを使っています

ロスを監視して一定回数（patience）以上ロスが下がらない場合，学習を終了する  
学習を早く終えるためではなく，過学習を防止するため  
そのため検証データのロスを監視する  

In [ ]:
#学習の実行
df_result = pd.DataFrame()
for i in range(len(batch_size_list)):
    # シード値の固定用
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(11)
    random.seed(101)
    tf.random.set_seed(1001)

    #モデル構築
    print('\n-- Model Construction ['+str(i+1)+'/'+str(len(batch_size_list))+'] --')
    model = Sequential(name='LSTM_'+str(batch_size_list[i]))
    model.add(LSTM(7, input_shape=(sequence_size, X_dim)))
    model.add(Dense(Y_dim))
    model.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.01))
    early_stopping = EarlyStopping(patience=10, monitor='val_loss', min_delta=0.01, restore_best_weights=True, verbose=0)
    print('Model:LSTM_'+str(batch_size_list[i]))

    #学習・検証
    print('> Train and Validation')
    history = model.fit(X_trainval_data, Y_trainval_data, batch_size=batch_size_list[i], epochs=epochs_max, validation_split=validation_split_rate, verbose=verbose_level, callbacks=[early_stopping])

    #学習・検証曲線
    make_graph(i, batch_size_list, history)

    #学習結果
    print('> Predict')
    pred_validation = model.predict(X_validation, verbose=0)
    pred_train = model.predict(X_train, verbose=0)

    train_score = pd.DataFrame([get_score(Y_train, pred_train)], columns=result_score, index=['Train_'+str(batch_size_list[i])])
    validation_score = pd.DataFrame([get_score(Y_validation, pred_validation)], columns=result_score, index=['Val_'+str(batch_size_list[i])])

    df_result = pd.concat([df_result, train_score, validation_score])
    r2_val, rmse_val = get_score(Y_validation, pred_validation)

    #ベストモデル更新判定
    print('> Best Model Judgement', end="")
    if(r2_val > best_r2):
        best_r2 = r2_val
        best_lstm_model = model
        print(': Update\n')
    else:
        print('\n')

print('\n-- Train and Validation Curve --')

※ 実行時間目安：46分  
バッチサイズが小さい場合も，理想に近いLossの推移をすることが多い（イテレーション数が多い，局所解に陥りにくいから？）

#### 評価

In [ ]:
# 学習結果表示
df_result

In [ ]:
#ベストモデル抽出
for j in batch_size_list:
    df_result.drop('Train_'+str(j), axis=0, inplace=True)
df_result[df_result['R^2'].idxmax():df_result['R^2'].idxmax()]

#### 訓練・検証予測線の描画

In [ ]:
#予測の実行
pred_train = best_lstm_model.predict(X_train, verbose=0)
pred_validation = best_lstm_model.predict(X_validation, verbose=0)

In [ ]:
#R^2に関するベストモデル(訓練データに対する予測グラフ)
cmap = plt.get_cmap('tab10')
plt.figure(figsize=[16.5,3.5])
plt.plot(range(0, len(Y_train)), Y_train[:,0], c=cmap(0), linewidth=1.5, label="True")
plt.plot(range(0, len(pred_train)), pred_train[:,0], c=cmap(1), linewidth=1.5, label="predict")
plt.title('Train Predict by "best_lstm_model"')
plt.xlabel('UNIX_Time Index')
plt.ylabel('Radiation')
plt.legend()
plt.show()

In [ ]:
#R^2に関するベストモデル(検証データに対する予測グラフ)
plt.figure(figsize=[16.5,3.5])
plt.plot(range(0, len(Y_validation)), Y_validation[:,0], c=cmap(0), linewidth=1.5, label="True")
plt.plot(range(0, len(pred_validation)), pred_validation[:,0], c=cmap(2), linewidth=1.5, label="predict")
plt.title('Validation Predict by "best_lstm_model"')
plt.xlabel('UNIX_Time Index')
plt.ylabel('Radiation')
plt.legend()
plt.show()

---

### さいごに

データを提供されたところから実際に機械学習で予測するところまでの一連の流れをやってみた  
当然だがこれはあくまで一例，思い付きで処理をしているだけなので各ステップで色々と考察すべきところがある  

時系列データは関係ないが，基本的にはここでやったようにウォーターフォール型で各ステップをこなしていくため，後戻りが面倒  
そのため、自分一人の考えで全てを進めていくのではなく，都度誰かに相談したり，進捗内容を関係者に共有したりしながら無駄のないように進めていくのが大事  
（といっても基本的に自分一人）  

何か研究テーマのヒントにでもなってれば幸いです

---

### 採点ポイント

評価時には, その人が実際に研究に取り組んだ時のことを想像されることを意識してください  
何をすれば評価されるというのは明確に決まっているわけではなく, 当然タイミング等の運要素もありますが, 基本的には相対的加点方式になると思います  

以下, 大まかな採点ポイント  

- 学習モデルの精度
- 進捗の量と質
- 進捗報告の質(量ではないことに注意)
- アイデア力
- 合理性と論理的な判断思考力

最終的な学習モデルの精度は言うまでもなく採点対象です  

進捗自体の量と質を正確に把握することは困難ですので, 毎週の進捗報告のスライドから判断せざるを得ません  
いかに, 限られた時間の中で自分のやったことを簡潔かつ正確に伝えることができるのかも非常に大事になってきます  
やったことを全てアピールしたい気持ちも分かりますが, 聞き手に配慮したスライド作りを期待しています  
スライドなんてとか思ってると, 共同研究のミーティングや対外発表等での発表時間制限で痛い目を見ます  

以下, ↓B4原田の当時の進捗スライドの一部↓

<img src='./image/image3.jpg' width=400>

<img src='./image/image4.jpg' width=400>

また, アイデアは結果の良し悪し関係なく, アイデアに裏付けられる合理的かつ論理的な思考過程があれば, 着眼点の鋭さを評価します  

「ここのサイトに書いてあったから, 真似したら精度が上がりました」「なんとなくうまくいくような感じがしたので」等の理由では聞き手も納得できないことは容易に想像できると思います  
しかし, 仮説を立てて実際に実験してみるという正しい手順を踏むと時間が足りないのも現実問題としてあります  
自分の出した進捗に対して, なぜうまくいったのかを調べたり, 考察を深めたりすることで, 進捗報告の際にはあたかも初めからそのような思考をしていて, 実際にその仮説を元にやってみると本当に精度が上がったという風に伝えることで聞き手も感心しますし, 合理的かつ論理的な判断思考力が評価されます  

まとめると, 約2ヵ月という限られた時間の中でなるべく効率的に要領良く取り組み, そこで生まれた結果を週に1回の数分(約5分)の間で聞き手に正確に伝えるということを心がけてください  
健闘を祈ります  